In [ ]:
import ipywidgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg, Heartbeat
from IPython.display import display
import threading
import time

# Controller 
controller = widgets.Controller(index=0)
display(controller)

# Robot 
robot = Robot()

# Camera 
cam_width = 640
cam_height = 480
camera = Camera.instance(width=cam_width, height=cam_height)
image = widgets.Image(format='jpeg', width=cam_width, height=cam_height)
display(image)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# Heartbeat stop 
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        robot.stop()

heartbeat = Heartbeat(period=0.5)
heartbeat.observe(handle_heartbeat_status, names='status')

# Arcade drive in background thread 
def arcade_loop():
    while True:
        if len(controller.axes) >= 2:
            deadzone = 0.05
            forward_scale = 0.8  # axis 1 mapped to [-0.8, 0.8]
            turn_scale    = 0.2  # axis 0 mapped to [-0.2, 0.2]
            
            forward = -controller.axes[1].value * forward_scale
            turn    = controller.axes[0].value * turn_scale

            if abs(forward) < deadzone:
                forward = 0
            if abs(turn) < deadzone:
                turn = 0

            left_motor_val = forward + turn
            right_motor_val = forward - turn

            left_motor  = max(-1, min(1, left_motor_val))
            right_motor = max(-1, min(1, right_motor_val))
            robot.left_motor.value  = left_motor
            robot.right_motor.value = right_motor
        time.sleep(0.02)  # 50 Hz update

thread = threading.Thread(target=arcade_loop, daemon=True)
thread.start()
